In [ ]:
using Pkg
Pkg.activate(".")

using Revise, PtyLab, TestImages, ImageShow, IndexFunArrays, FFTW, HDF5, Noise, FourierTools, CUDA
using EllipsisNotation
FFTW.set_num_threads(12)

## Object

In [ ]:
img_abs = Float32.(testimage("fabio_gray_512"))[200:400, 200:400]
img_phase = Float32.(testimage("resolution_test_512"))[200:400, 200:400]
object = img_abs .* cispi.(2 .* img_phase)

complex_show(object);

## Random Grid

In [ ]:
grid_size = size(object)
tile_size = (100, 100)

grr = PtyLab.grid_regular_rand(grid_size, tile_size, (15, 15), 30);
@show grr.overlap
show_grid(grr, only_points=false);

## Probe

In [ ]:
probe = IndexFunArrays.gaussian(Float32, tile_size, scale=0.010) .* cis.(Float32(2π) .* 
     4 .* gaussian(Float32, tile_size, scale=0.003));

complex_show(probe);

## Simulate and store as dataset

In [ ]:
ptychogram = zeros(Float32, (tile_size..., length(grr.tiles)));
p = Params()
o2d, d2o = Fraunhofer(probe, fftshiftFlag=true);

for (i, t) in enumerate(grr.tiles)
    ptychogram[:, :, i] = poisson(abs2.(o2d(view(object, t.i₁:t.i₂,  t.j₁:t.j₂) .* probe)), 1000)
end


lambda = 633f-9
z = 50f-3
dxd = 10f-6
Nd = size(ptychogram, 1)
dxo = lambda * z / (Nd * dxd)

fid_new = h5open("simulated_ptychography.hdf5", "w");
fid_new["Nd"] = Nd
fid_new["No"] = size(img_abs, 1)
fid_new["dxd"] = 10f-6
fid_new["encoder"] = PtyLab.encoder(grr, dxo, offset=(50, 50))
fid_new["wavelength"] = lambda
fid_new["entrancePupilDiameter"] = dxo * 30
fid_new["zo"] = z
fid_new["ptychogram"] = ptychogram
close(fid_new)
#@view_image ptychogram;

## Load dataset again

In [ ]:
experimentalData = ExperimentalDataCPM("simulated_ptychography.hdf5");

reconstruction = ReconstructionCPM(experimentalData, cuda=false);
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);
params2 = Params(fftshiftFlag = false, transposePtychogram = false, 
                 comStabilizationSwitch = false)

engine = PtyLab.ePIE(betaProbe = 0.75f0, betaObject = 0.75f0, numIterations = 50)

### On CPU

In [ ]:
reconstruction = ReconstructionCPM(experimentalData, cuda=false);
reconstruction = PtyLab.initializeObjectProbe!(reconstruction);

In [ ]:
@time p, o = PtyLab.reconstruct(engine, params2, reconstruction);

In [ ]:
complex_show(o[:, :, 1,1,1,1])

In [ ]:
complex_show(p[:, :, 1,1,1,1])